# 基礎編 解析ノートブック: 確率計画法の基本概念

このノートブックでは、CIGRE TB820に基づいた確率計画法の基本概念を実装し、問題1を詳細に解析します。

## 学習目標

1. 風力発電の出力変動を考慮した発電計画
2. 期待値最適化の実装
3. 確率分布の可視化と分析
4. 最適解の感度分析

In [ ]:
# 必要ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
import seaborn as sns
from IPython.display import display, Markdown

# 日本語フォント設定（警告を避けるため）
plt.rcParams['font.family'] = 'sans-serif'
plt.style.use('seaborn-v0_8')

# グリッドとサイズ設定
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['axes.grid'] = True

print("ライブラリのインポート完了")

## 問題設定

### 基本パラメータ

- **風力発電出力**: $W \sim N(50, 10^2)$ MW
- **従来発電機コスト**: 60 $/MWh
- **緊急電源コスト**: 200 $/MWh
- **電力需要**: 80 MW（固定）

### 最適化問題

$$
\min_{x} E[\text{Total Cost}] = 60x + 200 \cdot E[\max(80 - x - W, 0)]
$$

ここで $x$ は従来発電機の出力 [MW]

In [ ]:
# 問題パラメータの定義
# 風力発電の分布パラメータ
wind_mean = 50.0  # MW
wind_std = 10.0   # MW

# コストパラメータ
conv_cost = 60.0      # $/MWh
emergency_cost = 200.0 # $/MWh

# 電力需要
demand = 80.0  # MW

print(f"問題設定:")
print(f"風力発電: N({wind_mean}, {wind_std}²) MW")
print(f"従来発電コスト: {conv_cost} $/MWh")
print(f"緊急電源コスト: {emergency_cost} $/MWh")
print(f"電力需要: {demand} MW")

## 1. 風力発電出力の確率分布分析

In [ ]:
# 風力発電出力の範囲を設定
wind_range = np.linspace(wind_mean - 4*wind_std, wind_mean + 4*wind_std, 1000)

# 確率密度関数
wind_pdf = stats.norm.pdf(wind_range, wind_mean, wind_std)

# 累積分布関数
wind_cdf = stats.norm.cdf(wind_range, wind_mean, wind_std)

# 可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# PDF
ax1.plot(wind_range, wind_pdf, 'b-', linewidth=2, label=f'N({wind_mean}, {wind_std}²)')
ax1.axvline(wind_mean, color='r', linestyle='--', alpha=0.7, label=f'平均: {wind_mean}MW')
ax1.axvline(wind_mean - wind_std, color='orange', linestyle=':', alpha=0.7, label='±1σ')
ax1.axvline(wind_mean + wind_std, color='orange', linestyle=':', alpha=0.7)
ax1.fill_between(wind_range, wind_pdf, alpha=0.3)
ax1.set_xlabel('Wind Output [MW]')
ax1.set_ylabel('Probability Density')
ax1.set_title('Wind Output Distribution (PDF)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# CDF
ax2.plot(wind_range, wind_cdf, 'g-', linewidth=2)
ax2.axhline(0.5, color='r', linestyle='--', alpha=0.7, label='50%点')
ax2.axvline(wind_mean, color='r', linestyle='--', alpha=0.7)
ax2.set_xlabel('Wind Output [MW]')
ax2.set_ylabel('Cumulative Probability')
ax2.set_title('Wind Output Distribution (CDF)')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 統計量の計算
print(f"\n風力発電の統計量:")
print(f"平均: {wind_mean} MW")
print(f"標準偏差: {wind_std} MW")
print(f"95%信頼区間: [{wind_mean - 1.96*wind_std:.1f}, {wind_mean + 1.96*wind_std:.1f}] MW")

## 2. 期待コスト関数の定義と計算

In [ ]:
def expected_cost_analytical(x, wind_mean=50, wind_std=10, 
                           conv_cost=60, emergency_cost=200, demand=80):
    """
    解析的な期待コスト計算（正規分布の場合）
    
    Parameters:
    -----------
    x : float
        従来発電機の出力 [MW]
    
    Returns:
    --------
    float
        期待総コスト [$]
    """
    # 従来発電機のコスト
    conventional_cost = conv_cost * x
    
    # 不足分の閾値
    threshold = demand - x
    
    # 不足確率: P(W < threshold)
    shortage_prob = stats.norm.cdf(threshold, wind_mean, wind_std)
    
    # 条件付き期待不足量: E[threshold - W | W < threshold]
    if shortage_prob > 0:
        # 切断正規分布の期待値
        standardized_threshold = (threshold - wind_mean) / wind_std
        phi = stats.norm.pdf(standardized_threshold)
        Phi = stats.norm.cdf(standardized_threshold)
        
        if Phi > 1e-10:  # 数値安定性のため
            expected_shortage = (threshold - wind_mean) + wind_std * phi / Phi
        else:
            expected_shortage = 0
    else:
        expected_shortage = 0
    
    # 期待緊急電源コスト
    expected_emergency_cost = emergency_cost * shortage_prob * expected_shortage
    
    return conventional_cost + expected_emergency_cost


def expected_cost_simulation(x, n_samples=100000):
    """
    Monte Carlo シミュレーションによる期待コスト計算
    """
    # 風力発電出力をサンプリング
    wind_samples = np.random.normal(wind_mean, wind_std, n_samples)
    
    # 各サンプルでの不足電力量
    shortage = np.maximum(0, demand - x - wind_samples)
    
    # コスト計算
    conventional_cost = conv_cost * x
    emergency_costs = emergency_cost * shortage
    
    return conventional_cost + np.mean(emergency_costs)

# テスト
x_test = 30.0
cost_analytical = expected_cost_analytical(x_test)
cost_simulation = expected_cost_simulation(x_test)

print(f"従来発電 {x_test} MW の場合:")
print(f"解析解: ${cost_analytical:.2f}")
print(f"シミュレーション: ${cost_simulation:.2f}")
print(f"差異: ${abs(cost_analytical - cost_simulation):.2f}")

## 3. 期待コストの可視化と最適化

In [ ]:
# 従来発電機の出力範囲
x_range = np.linspace(0, 80, 100)

# 各出力での期待コストを計算
costs_analytical = [expected_cost_analytical(x) for x in x_range]

# 最適化（解析解）
result = optimize.minimize_scalar(expected_cost_analytical, bounds=(0, 100), method='bounded')
optimal_x = result.x
optimal_cost = result.fun

# 可視化
plt.figure(figsize=(12, 8))
plt.plot(x_range, costs_analytical, 'b-', linewidth=2, label='Expected Total Cost')
plt.axvline(optimal_x, color='r', linestyle='--', linewidth=2, 
            label=f'Optimal x = {optimal_x:.1f} MW')
plt.scatter([optimal_x], [optimal_cost], color='red', s=100, zorder=5)

# コスト成分の分析
conv_costs = conv_cost * x_range
emergency_costs = [expected_cost_analytical(x) - conv_cost*x for x in x_range]

plt.plot(x_range, conv_costs, '--', color='green', alpha=0.7, label='Conventional Cost')
plt.plot(x_range, emergency_costs, '--', color='orange', alpha=0.7, label='Expected Emergency Cost')

plt.xlabel('Conventional Generation [MW]')
plt.ylabel('Cost [$]')
plt.title('Expected Cost vs Conventional Generation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n最適化結果:")
print(f"最適な従来発電量: {optimal_x:.2f} MW")
print(f"最適期待コスト: ${optimal_cost:.2f}")
print(f"従来発電コスト: ${conv_cost * optimal_x:.2f}")
print(f"期待緊急電源コスト: ${optimal_cost - conv_cost * optimal_x:.2f}")

## 4. 感度分析

In [ ]:
# パラメータ感度分析
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# 1. 風力平均値の影響
wind_means = np.linspace(30, 70, 50)
optimal_x_wind_mean = []
optimal_costs_wind_mean = []

for wm in wind_means:
    result = optimize.minimize_scalar(
        lambda x: expected_cost_analytical(x, wind_mean=wm),
        bounds=(0, 100), method='bounded'
    )
    optimal_x_wind_mean.append(result.x)
    optimal_costs_wind_mean.append(result.fun)

ax1.plot(wind_means, optimal_x_wind_mean, 'b-', linewidth=2)
ax1.set_xlabel('Wind Mean [MW]')
ax1.set_ylabel('Optimal Conventional Gen [MW]')
ax1.set_title('Wind Mean Sensitivity')
ax1.grid(True, alpha=0.3)

# 2. 風力標準偏差の影響
wind_stds = np.linspace(5, 20, 50)
optimal_x_wind_std = []
optimal_costs_wind_std = []

for ws in wind_stds:
    result = optimize.minimize_scalar(
        lambda x: expected_cost_analytical(x, wind_std=ws),
        bounds=(0, 100), method='bounded'
    )
    optimal_x_wind_std.append(result.x)
    optimal_costs_wind_std.append(result.fun)

ax2.plot(wind_stds, optimal_x_wind_std, 'g-', linewidth=2)
ax2.set_xlabel('Wind Std [MW]')
ax2.set_ylabel('Optimal Conventional Gen [MW]')
ax2.set_title('Wind Uncertainty Sensitivity')
ax2.grid(True, alpha=0.3)

# 3. 緊急電源コストの影響
emergency_costs = np.linspace(100, 300, 50)
optimal_x_emergency = []
optimal_costs_emergency = []

for ec in emergency_costs:
    result = optimize.minimize_scalar(
        lambda x: expected_cost_analytical(x, emergency_cost=ec),
        bounds=(0, 100), method='bounded'
    )
    optimal_x_emergency.append(result.x)
    optimal_costs_emergency.append(result.fun)

ax3.plot(emergency_costs, optimal_x_emergency, 'r-', linewidth=2)
ax3.set_xlabel('Emergency Cost [$/MWh]')
ax3.set_ylabel('Optimal Conventional Gen [MW]')
ax3.set_title('Emergency Cost Sensitivity')
ax3.grid(True, alpha=0.3)

# 4. 需要の影響
demands = np.linspace(60, 100, 50)
optimal_x_demand = []
optimal_costs_demand = []

for d in demands:
    result = optimize.minimize_scalar(
        lambda x: expected_cost_analytical(x, demand=d),
        bounds=(0, 120), method='bounded'
    )
    optimal_x_demand.append(result.x)
    optimal_costs_demand.append(result.fun)

ax4.plot(demands, optimal_x_demand, 'm-', linewidth=2)
ax4.set_xlabel('Demand [MW]')
ax4.set_ylabel('Optimal Conventional Gen [MW]')
ax4.set_title('Demand Sensitivity')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 感度分析の要約
print("\n感度分析の結果:")
print(f"1. 風力平均が10MW増加 → 最適従来発電が約{(optimal_x_wind_mean[-1]-optimal_x_wind_mean[0])/(wind_means[-1]-wind_means[0])*10:.1f}MW減少")
print(f"2. 風力標準偏差が5MW増加 → 最適従来発電が約{(optimal_x_wind_std[-1]-optimal_x_wind_std[0])/(wind_stds[-1]-wind_stds[0])*5:.1f}MW増加")
print(f"3. 緊急電源コストが100$/MWh増加 → 最適従来発電が約{(optimal_x_emergency[-1]-optimal_x_emergency[0])/(emergency_costs[-1]-emergency_costs[0])*100:.1f}MW増加")
print(f"4. 需要が10MW増加 → 最適従来発電が約{(optimal_x_demand[-1]-optimal_x_demand[0])/(demands[-1]-demands[0])*10:.1f}MW増加")

## 5. リスク分析 - VaR と CVaR

In [ ]:
def calculate_cost_distribution(x, n_samples=10000):
    """
    特定の従来発電量でのコスト分布を計算
    """
    # 風力発電出力をサンプリング
    wind_samples = np.random.normal(wind_mean, wind_std, n_samples)
    
    # 各サンプルでの総コスト
    shortage = np.maximum(0, demand - x - wind_samples)
    total_costs = conv_cost * x + emergency_cost * shortage
    
    return total_costs

def calculate_var_cvar(costs, confidence_level=0.95):
    """
    VaRとCVaRを計算
    """
    sorted_costs = np.sort(costs)
    n = len(costs)
    
    # VaR
    var_index = int(confidence_level * n)
    var = sorted_costs[var_index]
    
    # CVaR
    cvar = np.mean(sorted_costs[var_index:])
    
    return var, cvar

# 最適解とサブ最適解での比較
x_scenarios = [optimal_x, optimal_x - 10, optimal_x + 10]
labels = ['Optimal', 'Conservative', 'Aggressive']
colors = ['blue', 'green', 'red']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

risk_results = []

for i, (x, label, color) in enumerate(zip(x_scenarios, labels, colors)):
    costs = calculate_cost_distribution(x)
    
    # ヒストグラム
    ax1.hist(costs, bins=50, alpha=0.6, label=f'{label} (x={x:.1f})', 
             color=color, density=True)
    
    # 統計量計算
    var_95, cvar_95 = calculate_var_cvar(costs, 0.95)
    var_99, cvar_99 = calculate_var_cvar(costs, 0.99)
    
    risk_results.append({
        'x': x,
        'label': label,
        'mean': np.mean(costs),
        'std': np.std(costs),
        'var_95': var_95,
        'cvar_95': cvar_95,
        'var_99': var_99,
        'cvar_99': cvar_99
    })

ax1.set_xlabel('Total Cost [$]')
ax1.set_ylabel('Probability Density')
ax1.set_title('Cost Distribution Comparison')
ax1.legend()
ax1.grid(True, alpha=0.3)

# リスク指標の比較
x_pos = np.arange(len(x_scenarios))
width = 0.35

vars_95 = [r['var_95'] for r in risk_results]
cvars_95 = [r['cvar_95'] for r in risk_results]

ax2.bar(x_pos - width/2, vars_95, width, label='VaR (95%)', alpha=0.7)
ax2.bar(x_pos + width/2, cvars_95, width, label='CVaR (95%)', alpha=0.7)

ax2.set_xlabel('Strategy')
ax2.set_ylabel('Risk Measure [$]')
ax2.set_title('Risk Measures Comparison')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(labels)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# リスク分析結果の表示
print("\n=== リスク分析結果 ===")
risk_df = pd.DataFrame(risk_results)
display(risk_df.round(2))

print(f"\n重要な観察:")
print(f"1. 最適解は期待コストを最小化するが、VaR/CVaRは中程度")
print(f"2. 保守的戦略はリスクを低減するが、期待コストが増加")
print(f"3. 積極的戦略は期待コストは低いが、高リスク")

## 6. 実践的考察とまとめ

### 主要な発見

1. **最適解の性質**: 風力発電の不確実性を考慮した最適な従来発電量は、需要と風力平均の中間値

2. **感度特性**:
   - 風力平均の増加 → 従来発電の減少
   - 風力不確実性の増加 → 従来発電の増加（リスク回避）
   - 緊急電源コストの増加 → 従来発電の増加

3. **リスク・リターン**: 期待コスト最適化とリスク管理のトレードオフ

### 実用的示唆

- 電力系統運用では、期待値だけでなくリスク指標も考慮が重要
- 風力発電の予測精度向上が運用コスト削減に大きく貢献
- 緊急時対応コストの削減も重要な改善ポイント

In [ ]:
# 最終的な推奨値の計算
print("=== 推奨運用戦略 ===")
print(f"\n基本シナリオ:")
print(f"最適従来発電量: {optimal_x:.1f} MW")
print(f"期待総コスト: ${optimal_cost:.2f}")
print(f"期待不足確率: {stats.norm.cdf(demand - optimal_x, wind_mean, wind_std):.1%}")

# 異なる信頼水準での推奨値
confidence_levels = [0.9, 0.95, 0.99]
print(f"\n信頼水準別推奨値:")
for conf in confidence_levels:
    # 不足確率がconf未満になる従来発電量
    required_x = demand - stats.norm.ppf(1-conf, wind_mean, wind_std)
    cost_at_required = expected_cost_analytical(required_x)
    print(f"信頼水準{conf:.0%}: {required_x:.1f} MW (コスト: ${cost_at_required:.2f})")

print(f"\n=== 分析完了 ===")
print(f"このノートブックでは、確率計画法の基本概念を実装し、")
print(f"風力発電の不確実性を考慮した発電計画問題を詳細に分析しました。")